In [1]:
import os

DIR = os.getcwd()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftConfig, PeftModel

model_card = "maywell/Synatra-42dot-1.3B"

config = PeftConfig.from_pretrained('output_dir')
model = AutoModelForCausalLM.from_pretrained(model_card)
tokenizer = AutoTokenizer.from_pretrained(model_card)
model = PeftModel.from_pretrained(model, 'output_dir')

In [6]:
from datasets import load_dataset

test_dataset = load_dataset('text', data_files={'test': os.path.join(DIR, 'data', 'test.txt')})

print(test_dataset)
from pprint import pprint
pprint(test_dataset['test']['text'][:5])

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 130
    })
})
['질문: 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요? 답변: ',
 '질문: 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요? 답변: ',
 '질문: 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요? 답변: ',
 '질문: 철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있을까요? 답변: ',
 '질문: 도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요? 답변: ']


In [34]:
import torch
model.eval()


with open("./output.txt", 'w') as f:
    with torch.no_grad():
        for line in test_dataset['test']['text']:
            inputs = tokenizer(line.strip(), return_tensors='pt')
            outputs = model.generate(input_ids=inputs['input_ids'], max_new_tokens=10)    
            
            outputs = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)
            
            q, a = outputs[0].split("답변: ", 1)
            
            f.write(f"{a}\n")
            